In [2]:
!pip install pdf2image pytesseract pdfplumber nltk


In [3]:
import os
from pdf2image import convert_from_path
import pytesseract
import pdfplumber
import nltk
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

text_folder_path = '/Users/somitachaudhari/Adv AI /Data/Text/' 



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/somitachaudhari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/somitachaudhari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
os.getcwd()

'/Users/somitachaudhari/Adv AI '

In [38]:

os.listdir('Data')


['.ipynb_checkpoints', 'Text']

In [37]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'\W', ' ', text)    
    text = text.lower()                

    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)


In [24]:
import pdfplumber
from PIL import Image
import pytesseract

def extract_text_with_ocr(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                # If no text, try OCR
                image = page.to_image()
                text = pytesseract.image_to_string(image.original)
            print(text)  # Display or save the text


In [26]:
def extract_text_from_pdf_ocr(pdf_path):
    text = ""
    pages = convert_from_path(pdf_path, dpi=300)
    for page_num, page_image in enumerate(pages):
        page_text = pytesseract.image_to_string(page_image)
        if page_text:
            text += page_text + "\n"
        else:
            print(f"Warning: No text found on page {page_num+1} of {pdf_path}")
    return text


In [28]:
# Test OCR on a single PDF page to confirm functionality
sample_pdf_path = '/Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning/1(a)-my_first perceptron-with-python.pdf'  
print(f"Testing OCR on a single page of: {sample_pdf_path}")
test_pages = convert_from_path(sample_pdf_path, dpi=300, first_page=21, last_page=1)
for page in test_pages:
    page_text = pytesseract.image_to_string(page)
    print("OCR Extracted Text from sample page:", page_text)


Testing OCR on a single page of: /Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning/1(a)-my_first perceptron-with-python.pdf


In [30]:
# Dictionary to store preprocessed text for each file
preprocessed_texts = {}

# Confirm directory traversal
print(f"Starting directory traversal in: {text_folder_path}")
found_files = False

for root, dirs, files in os.walk(text_folder_path):
    print(f"Checking directory: {root}")
    for file in files:
        if file.endswith('.pdf'):
            found_files = True
            file_path = os.path.join(root, file)
            print(f"Found PDF file: {file_path}")

            # Attempt text extraction with pdfplumber
            full_text = ""
            try:
                with pdfplumber.open(file_path) as pdf:
                    for page_num, page in enumerate(pdf.pages):
                        page_text = page.extract_text()
                        if page_text:
                            full_text += page_text
                        else:
                            print(f"Warning: No text found on page {page_num+1} of {file_path}")
            except Exception as e:
                print(f"Error processing with pdfplumber: {e}")
                full_text = ""  

            # Use OCR if pdfplumber extraction fails or is empty
            if not full_text.strip():  # Check if text is empty
                print(f"Falling back to OCR for {file_path}")
                full_text = extract_text_from_pdf_ocr(file_path)

            # Preprocess and store text if any was extracted
            if full_text.strip():
                cleaned_text = preprocess_text(full_text)
                preprocessed_texts[file] = cleaned_text
            else:
                print(f"No text extracted from {file_path}")

# Save the preprocessed text dictionary as a JSON file
if preprocessed_texts:
    with open('preprocessed_texts.json', 'w') as json_file:
        json.dump(preprocessed_texts, json_file)
    print("Preprocessed data saved successfully to preprocessed_texts.json")
else:
    print("No text data was processed. JSON file will be empty.")


Starting directory traversal in: /Users/somitachaudhari/Adv AI /Data/Text/
Checking directory: /Users/somitachaudhari/Adv AI /Data/Text/
Checking directory: /Users/somitachaudhari/Adv AI /Data/Text/Books
Checking directory: /Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning
Found PDF file: /Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning/2.Building-machine-learning-systems.pdf
Found PDF file: /Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning/9.Learning-scikit-learn-Machine-Learning-in-Python.pdf
Found PDF file: /Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning/5.Thoughtful-machine-learning-with-matthew-kirk5718(www.ebook-dl.com).pdf
Found PDF file: /Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning/1(a)-my_first perceptron-with-python.pdf
Found PDF file: /Users/somitachaudhari/Adv AI /Data/Text/Books/Machine Learning/15.Machine-Learning-The-Art-and-Science-of-Algorithms-that-Make-Sense-of-Data-Peter-Flach(www.ebook-dl